## Susceptible depletion and varying $R_{0}$
Building on the previous (`02suscept-deplete`) notebook,
we'll now look at varying the reproduction number with time,
because inferring the variation in this quantity is what
I'd like to achieve from these models.

As previously, the equation we're considering will be:
$\\i_t = (1-\frac{n_t}{N})R_t\sum_{\tau<t} i_{\tau}g_{t-\tau}$
However, now the $R_{t}$ value is determined both
by the proportion of the population remaining susceptible
and an external "random" process.
At this stage, the process will be arbitrary values,
and there are several functions that could be used 
at this stage (including a random walk and an 
autoregressive process).

In [ ]:
from typing import Dict
from scipy.stats import gamma
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'

### Parameters
Set model parameters, now including the population size.
Also get the generation times as previously.

In [ ]:
n_times = 40
seed = 1.0
r0 = 2.0
pop = 100.0
incidence = np.zeros(n_times)
incidence[0] = seed
gen_time_sd = 1.5
gen_time_mean = 5.0

def get_gamma_params_from_mean_sd(req_mean: float, req_sd: float) -> Dict[str, float]:
    var = req_sd ** 2.0
    scale = var / req_mean
    a = req_mean / scale
    return {'a': a, 'scale': scale}

gamma_params = get_gamma_params_from_mean_sd(gen_time_mean, gen_time_sd)
gen_time_densities = np.diff(gamma.cdf(range(n_times + 1), **gamma_params))

### Model run
Run the model with susceptible depletion,
and a variable intrinsic reproduction number.
Now we can manipulate the shape of the epicurve a little more.

In [ ]:
process_req = [2.0, 1.2, 2.4, 1.8]
process_times = np.linspace(0.0, n_times, len(process_req))
process_vals = np.interp(range(n_times), process_times, process_req)
suscept = pop - seed
for t in range(1, n_times):
    suscept_prop = suscept / pop
    infect_contribution_by_day = incidence[:t] * gen_time_densities[:t][::-1] * r0
    this_inc = infect_contribution_by_day.sum() * suscept_prop * process_vals[t]
    incidence[t] = this_inc
    suscept = max(suscept - this_inc, 0.0)
pd.Series(incidence).plot(labels={'index': 'day', 'value': 'incidence'})